In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Import dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cobra
import operator
import re

from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from scipy.stats import zscore

from sklearn.metrics.pairwise import pairwise_distances
from sklearn.decomposition import PCA

from src.gem.yeast8model import Yeast8Model

Construct model object, optimise

In [ ]:
glc_exch_rate = 16.89

wt = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")

Nutrient options

In [ ]:
# OPTION 1: Default: lots of glucose
wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

In [ ]:
# ALTERNATIVELY, OPTION 2: Custom glucose & ammonium exchange
glc_exch_rate = 0.194 * 8.6869
amm_exch_rate = 0.71 * 1.4848

wt.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)
wt.model.reactions.get_by_id("r_1654").bounds = (-amm_exch_rate, 0)
wt.model.reactions.get_by_id("r_1654_REV").bounds = (0, amm_exch_rate)

In [ ]:
# ALTERNATIVELY, OPTION 3: Custom pyruvate & ammonium exchange
pyr_exch_rate = 0.1
amm_exch_rate = 0.1

wt.model.reactions.get_by_id("r_1714").bounds = (0, 0)
wt.model.reactions.get_by_id("r_2033").bounds = (-pyr_exch_rate, 0)
wt.model.reactions.get_by_id("r_2033_REV").bounds = (0, pyr_exch_rate)
wt.model.reactions.get_by_id("r_1654").bounds = (-amm_exch_rate, 0)
wt.model.reactions.get_by_id("r_1654_REV").bounds = (0, amm_exch_rate)

Optimise and ablate

In [ ]:
wt.solution = wt.optimize()

# Ablate and store fluxes in each round
wt.ablation_result = wt.ablate()
ablation_fluxes = wt.ablation_fluxes

In [ ]:
ablation_fluxes

# Carbohydrates and proteins

## Research question: Does carbohydrate synthesis depend on protein synthesis?

Test by: Tell the cell to specialise in carbohydrates and then see if there is flux through any of the reactions that produce each of the 20 amino acids.

In [ ]:
# Get the amino acid metabolites
amino_acids = ['alanine', 'arginine', 'asparagine', 'aspartate', 'cysteine',
              'glutamine', 'glutamate', 'glycine', 'histidine', 'isoleucine',
              'leucine', 'lysine', 'methionine', 'phenylalanine', 'proline',
              'serine', 'threonine', 'tryptophan', 'tyrosine', 'valine']
amino_acids = np.array([f"L-{amino_acid} [cytoplasm]"
                        for amino_acid in amino_acids])

metabolite_names = np.array([metabolite.name
                    for metabolite in wt.model.metabolites])
metabolite_ids = np.array([metabolite.id
                  for metabolite in wt.model.metabolites])

amino_acid_index = np.nonzero(np.isin(metabolite_names, amino_acids))[0]
amino_metabolite_ids = metabolite_ids[amino_acid_index]

In [ ]:
# Create list of all reactions that involve the amino acids
amino_reaction_list = []
for amino_metabolite_id in amino_metabolite_ids:
    reactions_for_amino = list(
        wt.model.metabolites.get_by_id(amino_metabolite_id).reactions)
    amino_reaction_list = list(set(amino_reaction_list).union(reactions_for_amino))

In [ ]:
# Inspect fluxes through reactions (parallel case)
amino_reaction_ids = [reaction.id for reaction in amino_reaction_list]
wt.solution.fluxes.loc[amino_reaction_ids]

In [ ]:
# Get enzyme pseudoreactant for each reaction, if available
amino_enzyme_list = []
for reaction in amino_reaction_list:
    reactant_list = [metabolite.id for metabolite in reaction.reactants]
    new_enzyme_list = [enzyme for enzyme in reactant_list
                       if enzyme.startswith("prot_")]
    if new_enzyme_list:
        amino_enzyme_list.extend(new_enzyme_list)

In [ ]:
# Convert enzyme pseudoreactant IDs to enzyme usage flux IDs
amino_usage_flux_ids = [re.sub("prot_(\w+)\[c\]",
                         lambda x: f"draw_prot_{x.group(1)}", enzyme)
                  for enzyme in amino_enzyme_list]

In [ ]:
# See enzyme usages when the cell prioritises carbohydrate
ablation_fluxes["carbohydrate"].loc[amino_usage_flux_ids]

In [ ]:
# How many have a non-zero flux (within tolerance)?
print(sum(abs(ablation_fluxes["carbohydrate"].loc[amino_usage_flux_ids] > 1.11e-10)))
print(sum(abs(ablation_fluxes["protein"].loc[amino_usage_flux_ids] > 1.11e-10)))

> In a high-glucose environment, when the cell specialises in carbohydrate production, the cell does not re-allocate its proteome towards amino acid production.
>
> However, when the cell specialises in protein production (which happens at a different time as carbohydrate synthesis in sequential synthesis), the cell re-allocates some of its proteome towards amino acid metabolism.

## Research question: Does protein synthesis depend on carbohydrate synthesis?

Test by: Tell the cell to specialise in proteins and then see if there is flux through any of the reactions that produce each of the carbohydrates that contribute to biomass.

In [ ]:
# Get the carbohydrate metabolites
carbohydrates = [
    '(1->3)-beta-D-glucan [cell envelope]',
    '(1->6)-beta-D-glucan [cell envelope]',
    'glycogen [cytoplasm]',
    'mannan [cytoplasm]',
    'trehalose [cytoplasm]',
    'D-glucose [cytoplasm]',
]

carbohydrate_index = np.nonzero(np.isin(metabolite_names, carbohydrates))[0]
carb_metabolite_ids = metabolite_ids[carbohydrate_index]

In [ ]:
# Create list of all reactions that involve the carbohydrates
carb_reaction_list = []
for carb_metabolite_id in carb_metabolite_ids:
    reactions_for_carb = list(
        wt.model.metabolites.get_by_id(carb_metabolite_id).reactions)
    carb_reaction_list = list(set(carb_reaction_list).union(reactions_for_carb))

In [ ]:
# Inspect fluxes through reactions (parallel case)
carb_reaction_ids = [reaction.id for reaction in carb_reaction_list]
wt.solution.fluxes.loc[carb_reaction_ids]

In [ ]:
# Get enzyme pseudoreactant for each reaction, if available
carb_enzyme_list = []
for reaction in carb_reaction_list:
    reactant_list = [metabolite.id for metabolite in reaction.reactants]
    new_enzyme_list = [enzyme for enzyme in reactant_list
                       if enzyme.startswith("prot_")]
    if new_enzyme_list:
        carb_enzyme_list.extend(new_enzyme_list)

In [ ]:
carb_enzyme_list

In [ ]:
# Convert enzyme pseudoreactant IDs to enzyme usage flux IDs
carb_usage_flux_ids = [re.sub("prot_(\w+)\[c\]",
                         lambda x: f"draw_prot_{x.group(1)}", enzyme)
                  for enzyme in carb_enzyme_list]

In [ ]:
# See enzyme usages when the cell prioritises protein
ablation_fluxes["protein"].loc[carb_usage_flux_ids]

In [ ]:
# How many have a non-zero flux (within tolerance)?
print(sum(abs(ablation_fluxes["carbohydrate"].loc[carb_usage_flux_ids] > 1.11e-10)))
print(sum(abs(ablation_fluxes["protein"].loc[carb_usage_flux_ids] > 1.11e-10)))

> In a high-glucose environment, when the cell specialises in carbohydrate production, the cell re-allocates some of its proteome towards carbohydrate metabolism.
>
> However, when the cell specialises in protein production (which happens at a different time as carbohydrate synthesis in sequential synthesis), the cell does not re-allocate its proteome towards carbohydrate metabolism.

## Shared enzyme usage between carbohydrate production and protein production

In [ ]:
# Store enzyme usage reactions that are non-zero
# during pure carbohydrate synthesis
nonzero_in_carbohydrate = ablation_fluxes["carbohydrate"] > 1.11e-10
print(np.sum(nonzero_in_carbohydrate))

# Store enzyme usage reactions that are non-zero
# during pure protein synthesis
nonzero_in_protein = ablation_fluxes["protein"] > 1.11e-10
print(np.sum(nonzero_in_protein))

In [ ]:
# Find enzyme usage reactions that are non-zero
# in both pure carbohydrate synthesis and pure protein synthesis
nonzero_in_both = nonzero_in_carbohydrate & nonzero_in_protein
np.sum(nonzero_in_both)

In [ ]:
# Convert enzyme usage reaction IDs to metabolite IDs of corresponding enzymes
nonzero_in_both_list = nonzero_in_both[nonzero_in_both].index.to_list()
enzyme_list = [re.sub("draw_prot_(\w+)",
                     lambda x: f"prot_{x.group(1)}[c]", rxn)
              for rxn in nonzero_in_both_list]

In [ ]:
enzyme_list

You can use UNIPROT to map these protein IDs to genes & protein names.

In [ ]:
# Inspect flux of common reactions during protein synthesis
protein_usgs = ablation_fluxes["protein"]
protein_usgs[protein_usgs.index.isin(nonzero_in_both_list)]

In [ ]:
fluxes = protein_usgs[protein_usgs.index.isin(nonzero_in_both_list)]
print(np.min(fluxes))
print(np.max(fluxes))

In [ ]:
# Geometric mean
from scipy.stats import gmean
print(gmean(fluxes))